In [1]:
from dataclasses import dataclass, asdict
from pathlib import Path
import os
import json

import pyarrow as pa

from mp_blog.llm_utils import EXPECTED_SCHEMA

In [2]:
prompt_dir = Path(os.environ['PROJECT_ROOT']) / 'generative-question-answering'
prompt_dir.mkdir(exist_ok=True)

In [3]:
@dataclass
class PromptComponents:
    context: str
    question: str
    answer_format: str


PROMPT_TEMPLATE = """Context: {context}

Question: {question}

Answer format: {answer_format}

Answer:
"""

In [4]:
prompt_components = PromptComponents(
    context='In the time of chimpanzees, I was a monkey', 
    question='What was I in the time of chimpanzees',
    answer_format='Free text'
)

In [5]:
@dataclass
class PromptConfigurationVariants:
    """Semantically equivalent variants of prompt configuration components"""
    contexts: list
    questions: list
    answer_formats: list
    templates: list

In [6]:
expected_keys = ', '.join([field.name for field in EXPECTED_SCHEMA])


configurations_write = PromptConfigurationVariants(
    contexts=[
        """Quelle: https://www.dsv.de/schwimmen/wettkampf-national/bestenlisten/

Auswahl
Geschlecht:
M	W	X
Bahn:
25m	50m
Strecke:

100m Schmetterling
Zeitbereich:

Saison 2018/2019
Altersklasse:

AK 40 - JG 1974 - 1978
Punkte:

FINA 2022 (25m)
Region:

Deutschland


25 Einträge
Suche
Deutscher Rekord: 0:51,19 von Steffen Deibler (Hamburger SC r.V. von 1879) am 28.04.2013
Platz	Schwimmer	JG	Verein	Zeit	Punkte	Ort	Datum
1	Thomas Ehrhardt	1977	SSKC Poseidon Aschaffenburg	1:01,64	465	Gwangju	8/2019
2	Jochen Kaminski	1974	SSF Bonn 05	1:03,91	417	Karlsruhe	6/2019
3	Paul Larsen	1977	TSV Haar	1:05,01	397	Kranj	9/2018
4	Sebastian Kratzenstein	1978	BSC Robben	1:05,21	393	Karlsruhe	6/2019
5	Torben Kritzer	1977	Bad Homburger SC 1927	1:05,64	385	Karlsruhe	6/2019
6	Ralf Hildebrandt	1978	Dresdner SC 1898	1:06,13	377	Riesa	5/2019
7	Matthias Michaelsen	1977	SG Böhmetal	1:06,47	371	Braunschweig	3/2019
8	Lars Kellermann	1974	SG Badenweiler-Neuenburg	1:07,07	361	Karlsruhe	6/2019
9	Christian Hylla	1978	SSF Bonn 05	1:07,60	353	Kamen	6/2019
10	Kieran Garbutt	1976	SV Bayreuth	1:08,48	339	Pappenheim	7/2019


Quelle: https://www.dsv.de/schwimmen/wettkampf-national/bestenlisten/

Auswahl
Geschlecht:
M	W	X
Bahn:
25m	50m
Strecke:

100m Schmetterling
Zeitbereich:

Saison 2019/2020
Altersklasse:

AK 40 - JG 1975 - 1979
Punkte:

FINA 2022 (25m)
Region:

Deutschland


2 Einträge
Suche
Deutscher Rekord: 0:51,19 von Steffen Deibler (Hamburger SC r.V. von 1879) am 28.04.2013
Platz	Schwimmer	JG	Verein	Zeit	Punkte	Ort	Datum
1	Ralf Hildebrandt	1978	Dresdner SC 1898	1:05,66	385	Dresden	12/2019
2	Ivo Rudolph	1978	SV Grün-Weiß Wittenberg	1:12,48	286	Dresden	12/2019
""",
        "",
    ],
    questions=[
        "Was waren die top 5 Zeiten Deutschlands der Saison zwischen 2018 und 2020 der Männer auf 100m Schmetterling im Altersklasse 40-44?",
        "Was waren die top 5 Zeiten Deutschlands der Saison zwischen 2018 und 2020 der Maenner auf 100m Schmetterling im Altersklasse 40-44?",
        "What were the top 5 times in Germany between 2018 and 2020 for men in the 100m butterfly for the age group 40-44?"        
    ],
    answer_formats=[
        "Return a json document with keys `{expected_keys}`.".format(expected_keys=expected_keys),
        "Return only a json document as a list entries with keys `{expected_keys}`.".format(expected_keys=expected_keys)
        
    ],
    templates=[
        """Context: {context}

Question: {question}

Answer format: {answer_format}

Answer:
""",
        """Context: 


{context}

Question: {question}

Answer format: {answer_format}

Answer:
"""       
    ]
)

In [7]:
configurations_write = PromptConfigurationVariants(
    contexts=[
        """Quelle: https://www.dsv.de/schwimmen/wettkampf-national/bestenlisten/

Auswahl
Geschlecht:
M	W	X
Bahn:
25m	50m
Strecke:

100m Schmetterling
Zeitbereich:

Saison 2018/2019
Altersklasse:

AK 40 - JG 1974 - 1978
Punkte:

FINA 2022 (25m)
Region:

Deutschland


25 Einträge
Suche
Deutscher Rekord: 0:51,19 von Steffen Deibler (Hamburger SC r.V. von 1879) am 28.04.2013
Platz	Schwimmer	JG	Verein	Zeit	Punkte	Ort	Datum
1	Thomas Ehrhardt	1977	SSKC Poseidon Aschaffenburg	1:01,64	465	Gwangju	8/2019
2	Jochen Kaminski	1974	SSF Bonn 05	1:03,91	417	Karlsruhe	6/2019
3	Paul Larsen	1977	TSV Haar	1:05,01	397	Kranj	9/2018
4	Sebastian Kratzenstein	1978	BSC Robben	1:05,21	393	Karlsruhe	6/2019
5	Torben Kritzer	1977	Bad Homburger SC 1927	1:05,64	385	Karlsruhe	6/2019
6	Ralf Hildebrandt	1978	Dresdner SC 1898	1:06,13	377	Riesa	5/2019
7	Matthias Michaelsen	1977	SG Böhmetal	1:06,47	371	Braunschweig	3/2019
8	Lars Kellermann	1974	SG Badenweiler-Neuenburg	1:07,07	361	Karlsruhe	6/2019
9	Christian Hylla	1978	SSF Bonn 05	1:07,60	353	Kamen	6/2019
10	Kieran Garbutt	1976	SV Bayreuth	1:08,48	339	Pappenheim	7/2019


Quelle: https://www.dsv.de/schwimmen/wettkampf-national/bestenlisten/

Auswahl
Geschlecht:
M	W	X
Bahn:
25m	50m
Strecke:

100m Schmetterling
Zeitbereich:

Saison 2019/2020
Altersklasse:

AK 40 - JG 1975 - 1979
Punkte:

FINA 2022 (25m)
Region:

Deutschland


2 Einträge
Suche
Deutscher Rekord: 0:51,19 von Steffen Deibler (Hamburger SC r.V. von 1879) am 28.04.2013
Platz	Schwimmer	JG	Verein	Zeit	Punkte	Ort	Datum
1	Ralf Hildebrandt	1978	Dresdner SC 1898	1:05,66	385	Dresden	12/2019
2	Ivo Rudolph	1978	SV Grün-Weiß Wittenberg	1:12,48	286	Dresden	12/2019
""",
        "",
    ],
    questions=[
        "Was waren die top 5 Zeiten Deutschlands der Saison zwischen 2018 und 2020 der Männer auf 100m Schmetterling im Altersklasse 40-44?",
        "Was waren die top 5 Zeiten Deutschlands der Saison zwischen 2018 und 2020 der Maenner auf 100m Schmetterling im Altersklasse 40-44?",
        "What were the top 5 times in Germany between 2018 and 2020 for men in the 100m butterfly for the age group 40-44?"        
    ],
    answer_formats=[
        "Return a json document with keys `{expected_keys}`.".format(expected_keys=expected_keys),
        "Return only a json document as a list entries with keys `{expected_keys}`.".format(expected_keys=expected_keys)
        
    ],
    templates=[
        """Context: {context}

Question: {question}

Answer format: {answer_format}

Answer:
""",
        """Context: 


{context}

Question: {question}

Answer format: {answer_format}

Answer:
"""       
    ]
)

In [8]:
configurations_write_dict = asdict(configurations_write)
with open(prompt_dir / 'prompt_variants.json', 'w') as fp:
    json.dump(configurations_write_dict, fp, indent=2)

In [9]:
with open(prompt_dir / 'prompt_variants.json', 'r') as fp:
    configurations = json.load(fp)